In [ ]:
 from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# pip install cvxopt
%cd /content/drive/My\ Drive/Colab\ Notebooks/Students/Negin/

/content/drive/My Drive/code


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.neighbors import kneighbors_graph
from sklearn.metrics.cluster import adjusted_rand_score as ARI
from sklearn.metrics.cluster import normalized_mutual_info_score as NMI
from sklearn.cluster import SpectralClustering
from sklearn.metrics.pairwise import pairwise_distances
from sklearn import preprocessing as pre
import cvxpy as cp
import cvxopt as co

import warnings
warnings.filterwarnings('ignore')
from util import eval, loadDataset, normalization, kmeansInitialization, nnmf


def calcDist(X):
    dist=(pairwise_distances(X.T, X.T))
    return dist

eps = torch.tensor(10 ** -10)

datasets = ['3Sources.npy',
'BBCSport.npy',
'Caltech101.npy',
'Caltech_2.npy',
'Citeseer.npy',
'Coil100.npy',
'Cora.npy',
'EYaleB10.npy',
'Handwritten.npy',
'MNIST10.npy',
'UCIdigit.npy',
'Umist.npy',
'Yale32.npy',
'Yeast.npy',
'Cora2.npy',
'texas.npy',
'wisconsin.npy',
'washington.npy',
'cornell.npy',
'digit2.npy']


X0, Y, V, c = loadDataset(1)

co.solvers.options['show_progress'] = False


r = 20
k = 10

alpha = 0.1

beta = 0.01
lam = 0.04
iter = 20

sigma = torch.ones(V)/V
omega = torch.ones(V)/V

W = []
H = []
X = []
XXT = []
for v in range(V):

    # Normalizing Xs ???
    x_scale = pre.MinMaxScaler().fit_transform(X0[v].T).T

    X.append(torch.tensor(x_scale).type(torch.float32))

    XXT.append(X[v] @ X[v].T)
    d, n = X[v].shape


    W0, H0 = nnmf(X[v], r, iter)
    W.append(W0)
    H.append(H0.T)

F = torch.rand(n, c)

dist = [] # nodes distance matrices list
for Xv in X:
    dist.append(pairwise_distances(Xv.T, Xv.T))

dist_x = np.zeros((n, n))
for dist_v in dist:
    dist_x += dist_v / v

## init S
S = np.zeros((n, n))
cvx_P = co.matrix(np.eye(n))
cvx_G = co.matrix(0.0, (n, n))
cvx_G[::n + 1] = -1.0
cvx_A = co.matrix(1.0, (1, n))
cvx_b = co.matrix(1.0)
cvx_h = co.matrix(0.0, (n, 1))
for i in range(n):
    cvx_q = co.matrix(dist_x[i].T / (2 * alpha))
    solve = co.solvers.qp(cvx_P, cvx_q, cvx_G, cvx_h, cvx_A, cvx_b)
    S[i, :] = np.array(solve['x']).ravel()
## init S ended



S = torch.tensor(S).type(torch.float32)
D = torch.diag(torch.sum(S, dim = 0))
L = D - S

# Optimization
err = torch.zeros(iter)

for t in range(iter):

    for v in range(V):

        # Updating Ws
        Wn = (1 + beta) * (X[v] @ H[v])
        Wd = W[v] @ H[v].T @ H[v] + beta * (XXT[v] @ W[v])
        W[v] = W[v] * (Wn / torch.maximum(Wd, eps))

        # Updating Hs

        Hn = sigma[v] * (1 + beta) * (X[v].T @ W[v])         + omega[v] * (S @ H[v])
        Hd = sigma[v] * (H[v] @ W[v].T @ W[v] + beta * H[v]) + omega[v] * (D @ H[v])
        H[v] = H[v] * (Hn / torch.maximum(Hd, eps))

    # Updating S ?????????????????????????????????????????????????????????????????????????????????????????????????
    dist_f = calcDist(F)
    dist_x = torch.zeros((n, n))
    for i, dist_v in enumerate(dist):
        dist_x += dist_v * omega[i].numpy()
    dist_d = ((dist_x + lam * dist_f) / (2 * alpha)).numpy().astype(np.float64)

    for i in range(n):
        cvx_q = co.matrix(dist_d[i].T)
        solve = co.solvers.qp(cvx_P, cvx_q, cvx_G, cvx_h, cvx_A, cvx_b)
        S[i, :] = torch.tensor(np.array(solve['x']).ravel()).type(torch.float32)

    ## update S ended




    # VV = torch.zeros(n, n)
    # for vv in range(V):
    #     VV += omega[vv] * torch.cdist(H[vv], H[vv], p=2)**2
    # FF = lam * torch.cdist(F, F, p=2)**2
    # G = VV + FF

    # # D = torch.diag(torch.sum(G, dim=0))
    # etha = 1/k + (1/(2 * k * alpha))
    # S = - (1/(2 * alpha)) * G + etha
    # S [S<0] = 0

    # D = torch.diag(torch.sum(S, dim=0))

    # Updating F
    U, Si, VT = torch.svd(L)
    F = U[:,n-c:]

    # Updating Sigma
    for v in range(V):
        er = torch.norm(X[v] - W[v] @ H[v].T) **2 + beta * torch.norm(H[v].T - W[v].T @ X[v]) **2
        sigma[v] = 1/ torch.maximum(2 * (er ** 0.5), eps)
    # Updating Omega
    for v in range(V):
        er2 = torch.trace(H[v].T @ L @ H[v])
        omega[v] = 1/ torch.maximum(2 * (er2 ** 0.5), eps)


        # Calculating cost function
#         err[t] += torch.norm(X[v] - W[v] @ H[v]) ** 2 + gamma * torch.trace(H[v] @ (D[v] - A[v]) @ H[v].T)

# plt.plot(err)

# Spectral Clustering on the S matrix
pred = SpectralClustering(c, affinity='precomputed').fit(S).labels_

nmi, acc, ari, f1mi, f1ma, f1we, pur = eval(Y, pred)
nmi, acc, ari, f1mi, f1ma, f1we, pur

ModuleNotFoundError: No module named 'util'

In [ ]:
S

tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]])